In [1]:
%pip install langchain-community langchain-core
%pip install sklearn
%pip install pandas
%pip install nltk


You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import rdflib
import random
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
from langchain_community.graphs import OntotextGraphDBGraph
from constants import (
    TRAINED_MODEL_ID_PATH,
    TEST_DATA_PATH,
    SPARQL_FIX_PROMPT,
    QA_PROMPT,
    SPARQL_GENERATION_PROMPT,
)
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from tqdm import tqdm

In [3]:
class CustomOntotextGraphDBGraph(OntotextGraphDBGraph):
    def query(
        self,
        query: str,
    ) -> List[rdflib.query.ResultRow]:
        """
        Query the graph.
        """
        from rdflib.query import ResultRow

        res = self.graph.query(query)
        if res.type == "ASK":
            return [r for r in res if isinstance(r, bool)]
        return [r for r in res if isinstance(r, ResultRow)]

In [4]:
config = {
    "query_endpoint": "http://localhost:7200/repositories/imkg",
    "local_file": "/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/KG/iMKG.ttl",
    "local_file_format": "turtle",
}
try:
    graph = CustomOntotextGraphDBGraph(**config)
    print("Graph object created successfully.")
except Exception as e:
    print("Failed to create graph object:", e)

Graph object created successfully.


In [5]:
GPT4O_RESULTS_PATH = os.path.join(os.getcwd(), "data", "Gpt4o-qa_test_results.json")
QWEN_RESULTS_PATH = os.path.join(os.getcwd(), "data", "Qwen2.5-3B_qa_test_results.json")
LLAMA3_RESULTS_PATH = os.path.join(
    os.getcwd(), "data", "Llama3.2-3B-qa_test_results.json"
)

if os.path.exists(GPT4O_RESULTS_PATH):
    with open(GPT4O_RESULTS_PATH, "r", encoding="utf-8") as f:
        gpt4o_results = json.load(f)
else:
    gpt4o_results = []

if os.path.exists(QWEN_RESULTS_PATH):
    with open(QWEN_RESULTS_PATH, "r", encoding="utf-8") as f:
        qwen_results = json.load(f)[:106]
else:
    qwen_results = []

if os.path.exists(LLAMA3_RESULTS_PATH):
    with open(LLAMA3_RESULTS_PATH, "r", encoding="utf-8") as f:
        llama3_results = json.load(f)[:106]
else:
    llama3_results = []

# GPT-4o fine-tuning: 100 data points


In [6]:
# Calculate accuracy for GPT-4o results
correct_count_gpt4o = 0
for data_point in gpt4o_results:
    sparql_response = data_point.get("sparql_response")
    sample_query = data_point.get("sparql")

    if sparql_response and sample_query:
        try:
            generated_results = set(graph.query(sparql_response))
            sample_results = set(graph.query(sample_query))

            if generated_results == sample_results:
                correct_count_gpt4o += 1
        except Exception:
            continue

gpt4o_accuracy = correct_count_gpt4o / len(gpt4o_results) if gpt4o_results else 0
print(f"GPT-4o Accuracy: {gpt4o_accuracy:.2%}")

GPT-4o Accuracy: 89.62%


# Qwen2.5-3B fine-tuning: 100 data points


In [10]:
correct_count_qwen = 0

for data_point in qwen_results:
    sparql_response = data_point.get("generated_sparql")
    sample_query = data_point.get("sparql")

    if sparql_response and sample_query:
        try:
            generated_results = set(graph.query(sparql_response))
            sample_results = set(graph.query(sample_query))

            if generated_results == sample_results:
                correct_count_qwen += 1
        except Exception:
            continue

qwen_accuracy = correct_count_qwen / len(qwen_results) if qwen_results else 0
print(f"Qwen2.5-3B Accuracy: {qwen_accuracy:.2%}")

print("Processing complete! Results saved.")

Qwen2.5-3B Accuracy: 91.51%
Processing complete! Results saved.


# Llama3.2-3B fine-tuning


In [8]:
# Calculate accuracy for Llama3.2-3B results
correct_count_llama3 = 0

for data_point in llama3_results:
    sparql_response = data_point.get("sparql_response")
    sample_query = data_point.get("sparql")

    if sparql_response and sample_query:
        try:
            generated_results = set(graph.query(sparql_response))
            sample_results = set(graph.query(sample_query))

            if generated_results == sample_results:
                correct_count_llama3 += 1
        except Exception:
            continue

llama3_accuracy = correct_count_llama3 / len(llama3_results) if llama3_results else 0
print(f"Llama3.2-3B Accuracy: {llama3_accuracy:.2%}")

print("Processing complete! Results saved.")

Llama3.2-3B Accuracy: 84.91%
Processing complete! Results saved.
